In [1]:
# Import necessary packages
import polars as pl

In [2]:
# Load the bike trips and weather data
trips = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_trips.csv", null_values='NA')
weather = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_weather.csv", null_values=['NA',''])

trips = trips.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")) # ensure date column is typed correctly
weather = weather.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")) # ensure date column is typed correctly
print(trips.height)
print(weather.height)

78704
733


In [4]:
# Question 1, PART A:
trips_weather = trips.join(weather, on="date", how="inner")

print(trips_weather.head())
print(trips_weather.height)


shape: (5, 35)
┌──────┬──────────┬─────────────┬─────────────┬───┬─────────────┬────────┬────────────┬────────────┐
│ id   ┆ duration ┆ start_date  ┆ start_stati ┆ … ┆ cloud_cover ┆ events ┆ wind_dir_d ┆ zip_code_r │
│ ---  ┆ ---      ┆ ---         ┆ on_name     ┆   ┆ ---         ┆ ---    ┆ egrees     ┆ ight       │
│ i64  ┆ i64      ┆ str         ┆ ---         ┆   ┆ i64         ┆ str    ┆ ---        ┆ ---        │
│      ┆          ┆             ┆ str         ┆   ┆             ┆        ┆ i64        ┆ i64        │
╞══════╪══════════╪═════════════╪═════════════╪═══╪═════════════╪════════╪════════════╪════════════╡
│ 4721 ┆ 3        ┆ 2013-08-29T ┆ Market at   ┆ … ┆ 4           ┆ null   ┆ 286        ┆ 94107      │
│      ┆          ┆ 20:27:00Z   ┆ 10th        ┆   ┆             ┆        ┆            ┆            │
│ 4812 ┆ 3        ┆ 2013-08-29T ┆ 2nd at      ┆ … ┆ 4           ┆ null   ┆ 286        ┆ 94107      │
│      ┆          ┆ 21:30:00Z   ┆ Folsom      ┆   ┆             ┆        ┆  

In [5]:
# Question 1, PART B:
avg_bad_weather = (
    trips_weather
    .filter(pl.col("events").is_not_null())
    .select(pl.col("duration").mean().alias("avg_duration"))
)

print(avg_bad_weather)


shape: (1, 1)
┌──────────────┐
│ avg_duration │
│ ---          │
│ f64          │
╞══════════════╡
│ 10.34899     │
└──────────────┘


In [6]:
# Load baseball datasets
Batting = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/Batting.csv", null_values='NA')
People = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/People.csv", null_values='NA')
HallOfFame = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/HallOfFame.csv", null_values='NA')

In [7]:
# Question 2, PART A:
stats_w_bio = Batting.join(People, on="playerID", how="inner")

print(stats_w_bio.head())
print(stats_w_bio.height)


shape: (5, 47)
┌───────────┬────────┬───────┬────────┬───┬──────────┬───────────┬────────────┬────────────┐
│ playerID  ┆ yearID ┆ stint ┆ teamID ┆ … ┆ retroID  ┆ bbrefID   ┆ deathDate  ┆ birthDate  │
│ ---       ┆ ---    ┆ ---   ┆ ---    ┆   ┆ ---      ┆ ---       ┆ ---        ┆ ---        │
│ str       ┆ i64    ┆ i64   ┆ str    ┆   ┆ str      ┆ str       ┆ str        ┆ str        │
╞═══════════╪════════╪═══════╪════════╪═══╪══════════╪═══════════╪════════════╪════════════╡
│ abercda01 ┆ 1871   ┆ 1     ┆ TRO    ┆ … ┆ aberd101 ┆ abercda01 ┆ 1939-11-11 ┆ 1850-01-02 │
│ addybo01  ┆ 1871   ┆ 1     ┆ RC1    ┆ … ┆ addyb101 ┆ addybo01  ┆ 1910-04-09 ┆ null       │
│ allisar01 ┆ 1871   ┆ 1     ┆ CL1    ┆ … ┆ allia101 ┆ allisar01 ┆ 1916-02-25 ┆ 1849-01-29 │
│ allisdo01 ┆ 1871   ┆ 1     ┆ WS3    ┆ … ┆ allid101 ┆ allisdo01 ┆ 1916-12-19 ┆ 1846-07-12 │
│ ansonca01 ┆ 1871   ┆ 1     ┆ RC1    ┆ … ┆ ansoc101 ┆ ansonca01 ┆ 1922-04-14 ┆ 1852-04-17 │
└───────────┴────────┴───────┴────────┴───┴──────────┴─

In [8]:
# Question 2, PART B:
florida_hr = (
    stats_w_bio
    .filter(pl.col("birthState") == "FL")
    .select(pl.col("HR").sum().alias("total_hr"))
)

print(florida_hr)


shape: (1, 1)
┌──────────┐
│ total_hr │
│ ---      │
│ i64      │
╞══════════╡
│ 16225    │
└──────────┘


In [9]:
# Question 3, PART A:
hall_inducted = (
    HallOfFame
    .filter((pl.col("inducted") == "Y") & (pl.col("category") == "Player"))
    .select(["playerID"])
    .unique()
)

print(hall_inducted.head())
print(hall_inducted.height)


shape: (5, 1)
┌───────────┐
│ playerID  │
│ ---       │
│ str       │
╞═══════════╡
│ burkeje01 │
│ robinja02 │
│ piazzmi01 │
│ cronijo01 │
│ vaughar01 │
└───────────┘
270


In [11]:
# Question 3, PART B:
hof_hits = (
    Batting
    .join(hall_inducted, on="playerID", how="inner")
    .group_by("playerID")
    .agg(pl.col("H").sum().alias("career_hits"))
    .select(pl.col("career_hits").mean().alias("average_hits"))
)

print(hof_hits)

shape: (1, 1)
┌──────────────┐
│ average_hits │
│ ---          │
│ f64          │
╞══════════════╡
│ 1717.29918   │
└──────────────┘
